# Install required libraries if not already installed
!pip install wordcloud
!pip install seaborn
!pip install sklearn
!pip install transformers
!pip install torch
!pip install nltk

In [12]:
# Packages Installation
import pandas as pd
from transformers import Trainer, TrainingArguments, T5ForConditionalGeneration, T5Tokenizer
from datasets import load_metric
from sklearn.model_selection import train_test_split
import re

##### Fetching Metadata from CSV

In [119]:
Metadata = pd.read_csv('C:\\24592774_Dataset_Discovery_Using_LLM\\MetaData_Creation\\MetaData_Notebooks\\Enriched_MetaData_DataSet.csv')

In [120]:
Metadata.head()

,title,description,summary,tags,dataset_url,available_formats,initial_tags,augmented_tags,ner_tags,final_tags
0,"['2023w7', 'global', 'electric', 'vehicle', 'm...","['global', 'electric', 'vehicle', 'market', 's...","['editorsimple', 'original', 'visualization', ...","['makeover monday', 'cars', 'vehicles', 'elect...",https://data.world/makeovermonday/2023w7,['xlsx'],"source, data, original, vehicle, electric","original, electric, source, data, vehicle",NaN,"['original', ' electric', ' source', ' ', ' da..."
1,"['connected', 'electric', 'autonomous', 'vehic...","['connected', 'electric', 'autonomous', 'vehic...","['rolling', 'snapshot', 'data', 'collected', '...",['autonomous'],https://data.world/smartcolumbusos/650b7e59-af...,['csv'],"updated, available, including, vehicle, collec...","electric, source, data, available, including, ...",NaN,"['electric', ' ', ' source', ' data', ' availa..."
2,"['electric', 'vehicle', 'charging', 'stations']","['dataset', 'contains', 'session', 'details', ...","['dataset', 'contains', 'session', 'details', ...","['electric vehicle', 'environment', 'energy']",https://data.world/townofcary/electric-vehicle...,"['dbf', 'csv', 'shx', 'shp', 'json', 'prj']","dataset, charging, updated, available, source,...","electric, source, available, dataset, vehicle,...",NaN,"['electric', ' ', ' source', ' available', ' d..."
3,"['nyserda', 'electric', 'vehicle', 'drive', 'c...","['new', 'york', 'state', '’', 'charge', 'ny', ...","['original', 'title', 'nyserda', 'electric', '...","['ev', 'electric vehicle', 'bev', 'phev', 'ghg...",https://data.world/data-ny-gov/thd2-fu8y,['csv'],"service, using, use, information, research, sa...","original, electric, new, source, research, use...",NaN,"['original', ' electric', ' new', ' source', '..."
4,"['impact', 'uncoordinated', 'plugin', 'electri...","['impact', 'uncoordinated', 'plugin', 'electri...","['original', 'title', 'impact', 'uncoordinated...","['battery', 'consumption', 'data', 'energy', '...",https://data.world/us-doe-gov/8ae7e117-313b-40...,['xlsx'],"total, number, al, united, using, use, chargin...","original, number, source, electric, use, data,...",NaN,"['original', ' number', ' source', ' electric'..."


In [24]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tqdm import tqdm

In [50]:
MetaData_DS.dtypes

title                 object
description           object
summary               object
tags                  object
dataset_url           object
available_formats     object
initial_tags          object
augmented_tags        object
ner_tags             float64
final_tags            object
dtype: object

In [73]:
import torch
import torch.nn.functional as F
from transformers import BertTokenizer
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertModel, AdamW
import torch.nn as nn
import numpy as np
from sklearn.metrics import precision_recall_fscore_support

MetaData_DS = Metadata.copy()

# Initialize BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

max_length = 128  

# Tokenize dataset descriptions
tokenized_descriptions = tokenizer(list(MetaData_DS['description']), truncation=True, padding=True, max_length=max_length)

# Convert labels to numerical format
# For example, if you have binary labels (0 or 1)
labels = [1 if url else 0 for url in MetaData_DS['dataset_url']]

# Extract input_ids and attention_mask
input_ids = tokenized_descriptions['input_ids']
attention_mask = tokenized_descriptions['attention_mask']

# Split data into training and validation sets
train_ids, val_ids, train_masks, val_masks, train_labels, val_labels = train_test_split(input_ids, attention_mask, labels, test_size=0.2, random_state=42)

# Convert tokenized texts and labels to PyTorch tensors
train_ids = torch.tensor(train_ids)
val_ids = torch.tensor(val_ids)
train_masks = torch.tensor(train_masks)
val_masks = torch.tensor(val_masks)
train_labels = torch.tensor(train_labels)
val_labels = torch.tensor(val_labels)

# Create TensorDataset for training and validation sets
train_dataset = TensorDataset(train_ids, train_masks, train_labels)
val_dataset = TensorDataset(val_ids, val_masks, val_labels)

# Define batch size
batch_size = 16

# Create DataLoader for training and validation sets
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Define the BERT-based model
class BERTClassifier(nn.Module):
    def __init__(self, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(0.1)
        self.linear = nn.Linear(self.bert.config.hidden_size, num_classes)
    
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        logits = self.linear(pooled_output)
        return logits

# Instantiate the BERT model
num_classes = 2 
model = BERTClassifier(num_classes)

# Define the optimizer and loss function
optimizer = AdamW(model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()

# Training loop
epochs = 3
for epoch in range(epochs):
    model.train()  # Set the model to training mode
    total_loss = 0
    total_correct = 0
    total_samples = 0
    for batch in train_loader:
        inputs, attention_masks, labels = batch

        # Clear gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs, attention_masks)

        # Compute loss
        loss = criterion(outputs, labels)

        # Backpropagation
        loss.backward()

        # Update model parameters
        optimizer.step()

        # Accumulate the total loss
        total_loss += loss.item()

        # Calculate accuracy for this batch
        _, predicted = torch.max(outputs, dim=1)
        total_correct += (predicted == labels).sum().item()
        total_samples += labels.size(0)

    # Print average training loss for the epoch
    print(f"Epoch {epoch + 1}/{epochs}, Average Training Loss: {total_loss / len(train_loader)}")
    # Print training accuracy for the epoch
    print(f"Epoch {epoch + 1}/{epochs}, Training Accuracy: {total_correct / total_samples}")

# Validation loop
model.eval()  # Set the model to evaluation mode
total_correct = 0
total_samples = 0
all_predictions = []
all_labels = []
with torch.no_grad():
    for batch in val_loader:
        inputs, attention_masks, labels = batch

        # Forward pass
        outputs = model(inputs, attention_masks)
        
        # Compute predictions
        _, predicted = torch.max(outputs, dim=1)

        # Update total samples and total correct predictions
        total_samples += labels.size(0)
        total_correct += (predicted == labels).sum().item()

        # Save predictions and labels for metric calculation
        all_predictions.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Compute validation accuracy
accuracy = total_correct / total_samples
print(f"Validation Accuracy: {accuracy}")

# Compute other metrics
precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_predictions, average='binary')
print(f"Validation Precision: {precision}")
print(f"Validation Recall: {recall}")
print(f"Validation F1 Score: {f1}")


C:\Users\Steffi Grace\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\transformers\optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/3, Average Training Loss: 0.3679824247956276
Epoch 1/3, Training Accuracy: 0.9841269841269841
Epoch 2/3, Average Training Loss: 0.1431718897074461
Epoch 2/3, Training Accuracy: 1.0
Epoch 3/3, Average Training Loss: 0.066731839440763
Epoch 3/3, Training Accuracy: 1.0
Validation Accuracy: 1.0
Validation Precision: 1.0
Validation Recall: 1.0
Validation F1 Score: 1.0


The model seems to be overfitted

In [80]:
# Function to get BERT embeddings
def get_bert_embedding(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        outputs = model.bert(**inputs)
    return outputs.pooler_output

# Get BERT embeddings for all dataset descriptions
dataset_embeddings = [get_bert_embedding(desc, model, tokenizer) for desc in MetaData_DS['description']]
dataset_embeddings = torch.cat(dataset_embeddings, dim=0)

# Function to compute cosine similarity
def cosine_similarity(emb1, emb2):
    return F.cosine_similarity(emb1, emb2).mean().item()



In [81]:
# Function to predict dataset URLs based on search text
def predict_dataset_url(search_text):
    search_emb = get_bert_embedding(search_text, model, tokenizer)
    similarities = [cosine_similarity(search_emb, dataset_emb) for dataset_emb in dataset_embeddings]
    
    # Find the maximum and minimum similarity values
    max_similarity = max(similarities)
    min_similarity = min(similarities)
    
    # Calculate the 90% range
    range_90 = 0.90 * (max_similarity - min_similarity)
    
    # Find the threshold value
    threshold = min_similarity + range_90
    
    # Find the indices of datasets with similarity above the threshold
    similar_indices = [i for i, sim in enumerate(similarities) if sim > threshold]
    
    results = []
    for idx in similar_indices:
        dataset_info = {
            "title": MetaData_DS.iloc[idx]['title'],
            "dataset_url": MetaData_DS.iloc[idx]['dataset_url'],
            "cosine_similarity": similarities[idx]
        }
        results.append(dataset_info)
    
    return results

# Example usage
search_text = "electric vehicles"
results = predict_dataset_url(search_text)

# Display results
if not results:
    print("No datasets found above the 90% threshold.")
else:
    for result in results:
        print(f"Title: {result['title']}")
        print(f"Dataset URL: {result['dataset_url']}")
        print(f"Cosine Similarity: {result['cosine_similarity']}\n")


Title: ['gpm', 'ground', 'validation', 'twodimensional', 'video', 'disdromete', 'v1']
Dataset URL: https://data.world/us-nasa-gov/47e3817a-65aa-4af4-8d40-f0ed0f655bda
Cosine Similarity: 0.8438868522644043

Title: ['cars', 'purchased', 'vehicles', 'xls']
Dataset URL: https://data.world/nhtsa/cars-purchased-vehicles-xls
Cosine Similarity: 0.8458564877510071

Title: ['iptv', 'subscription', 'service', 'go']
Dataset URL: https://data.world/freemotion/which-iptv-subscription-service-should-you-go-for
Cosine Similarity: 0.8654544353485107

Title: ['recognized', 'sports']
Dataset URL: https://data.world/sports/recognized-sports
Cosine Similarity: 0.8654544353485107

Title: ['sports', 'illustrated', 'covers']
Dataset URL: https://data.world/crowdflower/sports-illustrated-covers
Cosine Similarity: 0.8654544353485107



In [89]:
import torch
import torch.nn.functional as F
from transformers import BertTokenizer
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertModel, AdamW
import torch.nn as nn
import numpy as np
from sklearn.metrics import precision_recall_fscore_support

MetaData_DS = Metadata.copy()

# Initialize BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

max_length = 128  

# Tokenize dataset descriptions
tokenized_descriptions = tokenizer(list(MetaData_DS['description']), truncation=True, padding=True, max_length=max_length)

# Convert labels to numerical format
# For example, if you have binary labels (0 or 1)
labels = [1 if url else 0 for url in MetaData_DS['dataset_url']]

# Extract input_ids and attention_mask
input_ids = tokenized_descriptions['input_ids']
attention_mask = tokenized_descriptions['attention_mask']

# Split data into training and validation sets
train_ids, val_ids, train_masks, val_masks, train_labels, val_labels = train_test_split(input_ids, attention_mask, labels, test_size=0.2, random_state=42)

# Convert tokenized texts and labels to PyTorch tensors
train_ids = torch.tensor(train_ids)
val_ids = torch.tensor(val_ids)
train_masks = torch.tensor(train_masks)
val_masks = torch.tensor(val_masks)
train_labels = torch.tensor(train_labels)
val_labels = torch.tensor(val_labels)

# Create TensorDataset for training and validation sets
train_dataset = TensorDataset(train_ids, train_masks, train_labels)
val_dataset = TensorDataset(val_ids, val_masks, val_labels)

# Define batch size
batch_size = 16

# Create DataLoader for training and validation sets
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Define the BERT-based model with dropout and L2 regularization
class BERTClassifier(nn.Module):
    def __init__(self, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(0.1)  # Add dropout layer
        self.linear = nn.Linear(self.bert.config.hidden_size, num_classes)
        self.regularization = nn.Linear(self.bert.config.hidden_size, self.bert.config.hidden_size)  # Add L2 regularization layer
    
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        logits = self.linear(pooled_output)
        # Add L2 regularization
        regularization_loss = torch.sum(torch.square(self.regularization.weight))
        return logits, regularization_loss

# Instantiate the BERT model
num_classes = 2 
model = BERTClassifier(num_classes)

# Define the optimizer and loss function
optimizer = AdamW(model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()

# Training loop
epochs = 3
for epoch in range(epochs):
    model.train()  # Set the model to training mode
    total_loss = 0
    total_correct = 0
    total_samples = 0
    for batch in train_loader:
        inputs, attention_masks, labels = batch

        # Clear gradients
        optimizer.zero_grad()

        # Forward pass
        outputs, regularization_loss = model(inputs, attention_masks)
        # Compute loss
        loss = criterion(outputs, labels)
        loss += 0.01 * regularization_loss  # Add regularization loss to the total loss

        # Backpropagation
        loss.backward()

        # Update model parameters
        optimizer.step()

        # Accumulate the total loss
        total_loss += loss.item()

        # Calculate accuracy for this batch
        _, predicted = torch.max(outputs, dim=1)
        total_correct += (predicted == labels).sum().item()
        total_samples += labels.size(0)

    # Print average training loss for the epoch
    print(f"Epoch {epoch + 1}/{epochs}, Average Training Loss: {total_loss / len(train_loader)}")
    # Print training accuracy for the epoch
    print(f"Epoch {epoch + 1}/{epochs}, Training Accuracy: {total_correct / total_samples}")

# Validation loop
model.eval()  # Set the model to evaluation mode
total_correct = 0
total_samples = 0
all_predictions = []
all_labels = []
with torch.no_grad():
    for batch in val_loader:
        inputs, attention_masks, labels = batch

        # Forward pass
        outputs, _ = model(inputs, attention_masks)
        
        # Compute predictions
        _, predicted = torch.max(outputs, dim=1)

        # Update total samples and total correct predictions
        total_samples += labels.size(0)
        total_correct += (predicted == labels).sum().item()

        # Save predictions and labels for metric calculation
        all_predictions.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Compute validation accuracy
accuracy = total_correct / total_samples
print(f"Validation Accuracy: {accuracy}")

# Compute other metrics
precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_predictions, average='binary')
print(f"Validation Precision: {precision}")
print(f"Validation Recall: {recall}")
print(f"Validation F1 Score: {f1}")


C:\Users\Steffi Grace\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\transformers\optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/3, Average Training Loss: 2.9691543579101562
Epoch 1/3, Training Accuracy: 0.8412698412698413
Epoch 2/3, Average Training Loss: 2.6702473759651184
Epoch 2/3, Training Accuracy: 1.0
Epoch 3/3, Average Training Loss: 2.6028189063072205
Epoch 3/3, Training Accuracy: 1.0
Validation Accuracy: 1.0
Validation Precision: 1.0
Validation Recall: 1.0
Validation F1 Score: 1.0


In [90]:
import torch
import torch.nn.functional as F
from transformers import BertTokenizer
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertModel, AdamW
import torch.nn as nn
from sklearn.metrics import precision_recall_fscore_support

MetaData_DS = Metadata.copy()

# Initialize BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

max_length = 128  

# Tokenize dataset descriptions
tokenized_descriptions = tokenizer(list(MetaData_DS['description']), truncation=True, padding=True, max_length=max_length)

# Convert labels to numerical format
# For example, if you have binary labels (0 or 1)
labels = [1 if url else 0 for url in MetaData_DS['dataset_url']]

# Extract input_ids and attention_mask
input_ids = tokenized_descriptions['input_ids']
attention_mask = tokenized_descriptions['attention_mask']

# Split data into training and validation sets
train_ids, val_ids, train_masks, val_masks, train_labels, val_labels = train_test_split(input_ids, attention_mask, labels, test_size=0.2, random_state=42)

# Convert tokenized texts and labels to PyTorch tensors
train_ids = torch.tensor(train_ids)
val_ids = torch.tensor(val_ids)
train_masks = torch.tensor(train_masks)
val_masks = torch.tensor(val_masks)
train_labels = torch.tensor(train_labels)
val_labels = torch.tensor(val_labels)

# Create TensorDataset for training and validation sets
train_dataset = TensorDataset(train_ids, train_masks, train_labels)
val_dataset = TensorDataset(val_ids, val_masks, val_labels)

# Define batch size
batch_size = 16

# Create DataLoader for training and validation sets
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Define the BERT-based model with dropout and L2 regularization
class BERTClassifier(nn.Module):
    def __init__(self, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(0.1)  # Add dropout layer
        self.linear = nn.Linear(self.bert.config.hidden_size, num_classes)
        self.regularization = nn.Linear(self.bert.config.hidden_size, self.bert.config.hidden_size)  # Add L2 regularization layer
    
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        logits = self.linear(pooled_output)
        # Add L2 regularization
        regularization_loss = torch.sum(torch.square(self.regularization.weight))
        return logits, regularization_loss

# Instantiate the BERT model
num_classes = 2  
model = BERTClassifier(num_classes)

# Define the optimizer and loss function
optimizer = AdamW(model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()

# Training loop
epochs = 3
for epoch in range(epochs):
    model.train()  # Set the model to training mode
    total_loss = 0
    total_correct = 0
    total_samples = 0
    for batch in train_loader:
        inputs, attention_masks, labels = batch

        # Clear gradients
        optimizer.zero_grad()

        # Forward pass
        outputs, regularization_loss = model(inputs, attention_masks)
        # Compute loss
        loss = criterion(outputs, labels)
        loss += 0.01 * regularization_loss  # Add regularization loss to the total loss

        # Backpropagation
        loss.backward()

        # Update model parameters
        optimizer.step()

        # Accumulate the total loss
        total_loss += loss.item()

        # Calculate accuracy for this batch
        _, predicted = torch.max(outputs, dim=1)
        total_correct += (predicted == labels).sum().item()
        total_samples += labels.size(0)

    # Print average training loss for the epoch
    print(f"Epoch {epoch + 1}/{epochs}, Average Training Loss: {total_loss / len(train_loader)}")
    # Print training accuracy for the epoch
    print(f"Epoch {epoch + 1}/{epochs}, Training Accuracy: {total_correct / total_samples}")

# Validation loop
model.eval()  # Set the model to evaluation mode
total_correct = 0
total_samples = 0
all_predictions = []
all_labels = []
with torch.no_grad():
    for batch in val_loader:
        inputs, attention_masks, labels = batch

        # Forward pass
        outputs, _ = model(inputs, attention_masks)
        
        # Compute predictions
        _, predicted = torch.max(outputs, dim=1)

        # Update total samples and total correct predictions
        total_samples += labels.size(0)
        total_correct += (predicted == labels).sum().item()

        # Save predictions and labels for metric calculation
        all_predictions.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Compute validation accuracy
accuracy = total_correct / total_samples
print(f"Validation Accuracy: {accuracy}")

# Compute other metrics
precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_predictions, average='binary')
print(f"Validation Precision: {precision}")
print(f"Validation Recall: {recall}")
print(f"Validation F1 Score: {f1}")


C:\Users\Steffi Grace\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\transformers\optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/3, Average Training Loss: 3.144028604030609
Epoch 1/3, Training Accuracy: 0.746031746031746
Epoch 2/3, Average Training Loss: 2.8386403918266296
Epoch 2/3, Training Accuracy: 1.0
Epoch 3/3, Average Training Loss: 2.667064368724823
Epoch 3/3, Training Accuracy: 1.0
Validation Accuracy: 1.0
Validation Precision: 1.0
Validation Recall: 1.0
Validation F1 Score: 1.0


In [91]:
import torch
import torch.nn.functional as F
from transformers import BertTokenizer
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertModel
import torch.nn as nn
from sklearn.metrics import precision_recall_fscore_support
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau

MetaData_DS = Metadata.copy()

# Initialize BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

max_length = 128 

# Tokenize dataset descriptions
tokenized_descriptions = tokenizer(list(MetaData_DS['description']), truncation=True, padding=True, max_length=max_length)

# Convert labels to numerical format
labels = [1 if url else 0 for url in MetaData_DS['dataset_url']]

# Extract input_ids and attention_mask
input_ids = tokenized_descriptions['input_ids']
attention_mask = tokenized_descriptions['attention_mask']

# Split data into training and validation sets
train_ids, val_ids, train_masks, val_masks, train_labels, val_labels = train_test_split(input_ids, attention_mask, labels, test_size=0.2, random_state=42)

# Convert tokenized texts and labels to PyTorch tensors
train_ids = torch.tensor(train_ids)
val_ids = torch.tensor(val_ids)
train_masks = torch.tensor(train_masks)
val_masks = torch.tensor(val_masks)
train_labels = torch.tensor(train_labels)
val_labels = torch.tensor(val_labels)

# Create TensorDataset for training and validation sets
train_dataset = TensorDataset(train_ids, train_masks, train_labels)
val_dataset = TensorDataset(val_ids, val_masks, val_labels)

# Define batch size
batch_size = 16

# Create DataLoader for training and validation sets
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Define the BERT-based model with dropout and L2 regularization
class BERTClassifier(nn.Module):
    def __init__(self, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(0.3)  # Increase dropout rate
        self.linear = nn.Linear(self.bert.config.hidden_size, num_classes)
    
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        logits = self.linear(pooled_output)
        return logits

# Instantiate the BERT model
num_classes = 2  
model = BERTClassifier(num_classes)

# Define the optimizer, scheduler and loss function
optimizer = AdamW(model.parameters(), lr=2e-5)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=1, verbose=True)
criterion = nn.CrossEntropyLoss()

# Training loop with early stopping
epochs = 10
patience = 2
best_loss = float('inf')
trigger_times = 0

for epoch in range(epochs):
    model.train()  # Set the model to training mode
    total_loss = 0
    total_correct = 0
    total_samples = 0
    for batch in train_loader:
        inputs, attention_masks, labels = batch

        # Clear gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs, attention_masks)
        # Compute loss
        loss = criterion(outputs, labels)

        # Backpropagation
        loss.backward()

        # Update model parameters
        optimizer.step()

        # Accumulate the total loss
        total_loss += loss.item()

        # Calculate accuracy for this batch
        _, predicted = torch.max(outputs, dim=1)
        total_correct += (predicted == labels).sum().item()
        total_samples += labels.size(0)

    avg_train_loss = total_loss / len(train_loader)
    train_accuracy = total_correct / total_samples

    # Validation loop
    model.eval()  # Set the model to evaluation mode
    val_loss = 0
    total_correct = 0
    total_samples = 0
    all_predictions = []
    all_labels = []
    with torch.no_grad():
        for batch in val_loader:
            inputs, attention_masks, labels = batch

            # Forward pass
            outputs = model(inputs, attention_masks)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            
            # Compute predictions
            _, predicted = torch.max(outputs, dim=1)

            # Update total samples and total correct predictions
            total_samples += labels.size(0)
            total_correct += (predicted == labels).sum().item()

            # Save predictions and labels for metric calculation
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_val_loss = val_loss / len(val_loader)
    accuracy = total_correct / total_samples
    scheduler.step(avg_val_loss)
    
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_predictions, average='binary')

    print(f"Epoch {epoch + 1}/{epochs}")
    print(f"Train Loss: {avg_train_loss}, Train Accuracy: {train_accuracy}")
    print(f"Validation Loss: {avg_val_loss}, Validation Accuracy: {accuracy}")
    print(f"Validation Precision: {precision}, Validation Recall: {recall}, Validation F1 Score: {f1}")

    # Early stopping
    if avg_val_loss < best_loss:
        best_loss = avg_val_loss
        trigger_times = 0
    else:
        trigger_times += 1
        if trigger_times >= patience:
            print('Early stopping!')
            break


Epoch 1/10
Train Loss: 0.5783726349473, Train Accuracy: 0.7301587301587301
Validation Loss: 0.3101840615272522, Validation Accuracy: 1.0
Validation Precision: 1.0, Validation Recall: 1.0, Validation F1 Score: 1.0
Epoch 2/10
Train Loss: 0.2463531270623207, Train Accuracy: 1.0
Validation Loss: 0.15873892605304718, Validation Accuracy: 1.0
Validation Precision: 1.0, Validation Recall: 1.0, Validation F1 Score: 1.0
Epoch 3/10
Train Loss: 0.14758636429905891, Train Accuracy: 1.0
Validation Loss: 0.08207385241985321, Validation Accuracy: 1.0
Validation Precision: 1.0, Validation Recall: 1.0, Validation F1 Score: 1.0
Epoch 4/10
Train Loss: 0.07775966823101044, Train Accuracy: 1.0
Validation Loss: 0.04271232709288597, Validation Accuracy: 1.0
Validation Precision: 1.0, Validation Recall: 1.0, Validation F1 Score: 1.0
Epoch 5/10
Train Loss: 0.041524085216224194, Train Accuracy: 1.0
Validation Loss: 0.023756233975291252, Validation Accuracy: 1.0
Validation Precision: 1.0, Validation Recall: 1.0,

In [92]:
import torch
import torch.nn.functional as F
from transformers import BertTokenizer
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertModel
import torch.nn as nn
from sklearn.metrics import precision_recall_fscore_support
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau

MetaData_DS = Metadata.copy()

# Initialize BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


max_length = 128  

# Tokenize dataset descriptions
tokenized_descriptions = tokenizer(list(MetaData_DS['description']), truncation=True, padding=True, max_length=max_length)

# Convert labels to numerical format
labels = [1 if url else 0 for url in MetaData_DS['dataset_url']]

# Extract input_ids and attention_mask
input_ids = tokenized_descriptions['input_ids']
attention_mask = tokenized_descriptions['attention_mask']

# Split data into training and validation sets
train_ids, val_ids, train_masks, val_masks, train_labels, val_labels = train_test_split(input_ids, attention_mask, labels, test_size=0.2, random_state=42)

# Convert tokenized texts and labels to PyTorch tensors
train_ids = torch.tensor(train_ids)
val_ids = torch.tensor(val_ids)
train_masks = torch.tensor(train_masks)
val_masks = torch.tensor(val_masks)
train_labels = torch.tensor(train_labels)
val_labels = torch.tensor(val_labels)

# Create TensorDataset for training and validation sets
train_dataset = TensorDataset(train_ids, train_masks, train_labels)
val_dataset = TensorDataset(val_ids, val_masks, val_labels)

# Define batch size
batch_size = 16

# Create DataLoader for training and validation sets
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Define the BERT-based model with dropout and L2 regularization
class BERTClassifier(nn.Module):
    def __init__(self, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased', hidden_dropout_prob=0.3, attention_probs_dropout_prob=0.3) 
        self.dropout = nn.Dropout(0.3)  # Increase dropout rate
        self.linear = nn.Linear(self.bert.config.hidden_size, num_classes)
    
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        logits = self.linear(pooled_output)
        return logits

# Instantiate the BERT model
num_classes = 2  
model = BERTClassifier(num_classes)

# Define the optimizer with weight decay
optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)  # Add weight decay
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=1, verbose=True)
criterion = nn.CrossEntropyLoss()

# Training loop with early stopping
epochs = 10
patience = 2
best_loss = float('inf')
trigger_times = 0

for epoch in range(epochs):
    model.train()  # Set the model to training mode
    total_loss = 0
    total_correct = 0
    total_samples = 0
    for batch in train_loader:
        inputs, attention_masks, labels = batch

        # Clear gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs, attention_masks)
        # Compute loss
        loss = criterion(outputs, labels)

        # Backpropagation
        loss.backward()

        # Update model parameters
        optimizer.step()

        # Accumulate the total loss
        total_loss += loss.item()

        # Calculate accuracy for this batch
        _, predicted = torch.max(outputs, dim=1)
        total_correct += (predicted == labels).sum().item()
        total_samples += labels.size(0)

    avg_train_loss = total_loss / len(train_loader)
    train_accuracy = total_correct / total_samples

    # Validation loop
    model.eval()  # Set the model to evaluation mode
    val_loss = 0
    total_correct = 0
    total_samples = 0
    all_predictions = []
    all_labels = []
    with torch.no_grad():
        for batch in val_loader:
            inputs, attention_masks, labels = batch

            # Forward pass
            outputs = model(inputs, attention_masks)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            
            # Compute predictions
            _, predicted = torch.max(outputs, dim=1)

            # Update total samples and total correct predictions
            total_samples += labels.size(0)
            total_correct += (predicted == labels).sum().item()

            # Save predictions and labels for metric calculation
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_val_loss = val_loss / len(val_loader)
    accuracy = total_correct / total_samples
    scheduler.step(avg_val_loss)
    
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_predictions, average='binary')

    print(f"Epoch {epoch + 1}/{epochs}")
    print(f"Train Loss: {avg_train_loss}, Train Accuracy: {train_accuracy}")
    print(f"Validation Loss: {avg_val_loss}, Validation Accuracy: {accuracy}")
    print(f"Validation Precision: {precision}, Validation Recall: {recall}, Validation F1 Score: {f1}")

    # Early stopping
    if avg_val_loss < best_loss:
        best_loss = avg_val_loss
        trigger_times = 0
    else:
        trigger_times += 1
        if trigger_times >= patience:
            print('Early stopping!')
            break


Epoch 1/10
Train Loss: 0.4851943999528885, Train Accuracy: 0.9047619047619048
Validation Loss: 0.2962067127227783, Validation Accuracy: 1.0
Validation Precision: 1.0, Validation Recall: 1.0, Validation F1 Score: 1.0
Epoch 2/10
Train Loss: 0.30162081867456436, Train Accuracy: 1.0
Validation Loss: 0.1883525401353836, Validation Accuracy: 1.0
Validation Precision: 1.0, Validation Recall: 1.0, Validation F1 Score: 1.0
Epoch 3/10
Train Loss: 0.18902996182441711, Train Accuracy: 1.0
Validation Loss: 0.11559051275253296, Validation Accuracy: 1.0
Validation Precision: 1.0, Validation Recall: 1.0, Validation F1 Score: 1.0
Epoch 4/10
Train Loss: 0.11456568539142609, Train Accuracy: 1.0
Validation Loss: 0.06862010806798935, Validation Accuracy: 1.0
Validation Precision: 1.0, Validation Recall: 1.0, Validation F1 Score: 1.0
Epoch 5/10
Train Loss: 0.07565614953637123, Train Accuracy: 1.0
Validation Loss: 0.03835596516728401, Validation Accuracy: 1.0
Validation Precision: 1.0, Validation Recall: 1.0

In [93]:
import torch
import torch.nn.functional as F
from transformers import BertTokenizer
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertModel
import torch.nn as nn
from sklearn.metrics import precision_recall_fscore_support
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau

MetaData_DS = Metadata.copy()

# Initialize BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

max_length = 128  

# Tokenize dataset descriptions
tokenized_descriptions = tokenizer(list(MetaData_DS['description']), truncation=True, padding=True, max_length=max_length)

# Convert labels to numerical format
labels = [1 if url else 0 for url in MetaData_DS['dataset_url']]

# Extract input_ids and attention_mask
input_ids = tokenized_descriptions['input_ids']
attention_mask = tokenized_descriptions['attention_mask']

# Split data into training and validation sets
train_ids, val_ids, train_masks, val_masks, train_labels, val_labels = train_test_split(input_ids, attention_mask, labels, test_size=0.2, random_state=42)

# Convert tokenized texts and labels to PyTorch tensors
train_ids = torch.tensor(train_ids)
val_ids = torch.tensor(val_ids)
train_masks = torch.tensor(train_masks)
val_masks = torch.tensor(val_masks)
train_labels = torch.tensor(train_labels)
val_labels = torch.tensor(val_labels)

# Create TensorDataset for training and validation sets
train_dataset = TensorDataset(train_ids, train_masks, train_labels)
val_dataset = TensorDataset(val_ids, val_masks, val_labels)

# Define batch size
batch_size = 16

# Create DataLoader for training and validation sets
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Define the BERT-based model with increased dropout
class BERTClassifier(nn.Module):
    def __init__(self, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased', hidden_dropout_prob=0.5, attention_probs_dropout_prob=0.5)  # Further increased BERT dropout
        self.dropout = nn.Dropout(0.5)  # Further increased dropout rate
        self.linear = nn.Linear(self.bert.config.hidden_size, num_classes)
    
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        logits = self.linear(pooled_output)
        return logits

# Instantiate the BERT model
num_classes = 2 
model = BERTClassifier(num_classes)

# Define the optimizer with weight decay and gradient clipping
optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=1, verbose=True)
criterion = nn.CrossEntropyLoss()

# Training loop with early stopping
epochs = 10
patience = 2
best_loss = float('inf')
trigger_times = 0

for epoch in range(epochs):
    model.train()  # Set the model to training mode
    total_loss = 0
    total_correct = 0
    total_samples = 0
    for batch in train_loader:
        inputs, attention_masks, labels = batch

        # Clear gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs, attention_masks)
        # Compute loss
        loss = criterion(outputs, labels)

        # Backpropagation
        loss.backward()

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        # Update model parameters
        optimizer.step()

        # Accumulate the total loss
        total_loss += loss.item()

        # Calculate accuracy for this batch
        _, predicted = torch.max(outputs, dim=1)
        total_correct += (predicted == labels).sum().item()
        total_samples += labels.size(0)

    avg_train_loss = total_loss / len(train_loader)
    train_accuracy = total_correct / total_samples

    # Validation loop
    model.eval()  # Set the model to evaluation mode
    val_loss = 0
    total_correct = 0
    total_samples = 0
    all_predictions = []
    all_labels = []
    with torch.no_grad():
        for batch in val_loader:
            inputs, attention_masks, labels = batch

            # Forward pass
            outputs = model(inputs, attention_masks)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            
            # Compute predictions
            _, predicted = torch.max(outputs, dim=1)

            # Update total samples and total correct predictions
            total_samples += labels.size(0)
            total_correct += (predicted == labels).sum().item()

            # Save predictions and labels for metric calculation
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_val_loss = val_loss / len(val_loader)
    accuracy = total_correct / total_samples
    scheduler.step(avg_val_loss)
    
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_predictions, average='binary')

    print(f"Epoch {epoch + 1}/{epochs}")
    print(f"Train Loss: {avg_train_loss}, Train Accuracy: {train_accuracy}")
    print(f"Validation Loss: {avg_val_loss}, Validation Accuracy: {accuracy}")
    print(f"Validation Precision: {precision}, Validation Recall: {recall}, Validation F1 Score: {f1}")

    # Early stopping
    if avg_val_loss < best_loss:
        best_loss = avg_val_loss
        trigger_times = 0
    else:
        trigger_times += 1
        if trigger_times >= patience:
            print('Early stopping!')
            break


Epoch 1/10
Train Loss: 0.8908563852310181, Train Accuracy: 0.2698412698412698
Validation Loss: 0.7927955389022827, Validation Accuracy: 0.125
Validation Precision: 1.0, Validation Recall: 0.125, Validation F1 Score: 0.2222222222222222
Epoch 2/10
Train Loss: 0.39846061170101166, Train Accuracy: 0.9523809523809523
Validation Loss: 0.4657982289791107, Validation Accuracy: 1.0
Validation Precision: 1.0, Validation Recall: 1.0, Validation F1 Score: 1.0
Epoch 3/10
Train Loss: 0.28323177620768547, Train Accuracy: 1.0
Validation Loss: 0.25540706515312195, Validation Accuracy: 1.0
Validation Precision: 1.0, Validation Recall: 1.0, Validation F1 Score: 1.0
Epoch 4/10
Train Loss: 0.19210970029234886, Train Accuracy: 1.0
Validation Loss: 0.17330004274845123, Validation Accuracy: 1.0
Validation Precision: 1.0, Validation Recall: 1.0, Validation F1 Score: 1.0
Epoch 5/10
Train Loss: 0.15426048263907433, Train Accuracy: 1.0
Validation Loss: 0.13986526429653168, Validation Accuracy: 1.0
Validation Prec

In [95]:
import torch
import torch.nn.functional as F
from transformers import BertTokenizer
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertModel
import torch.nn as nn
from sklearn.metrics import precision_recall_fscore_support
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau

MetaData_DS = Metadata.copy()

# Initialize BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

max_length = 128  
MetaData_DS['combined_text'] = MetaData_DS['title'].astype(str) + ' ' + \
                               MetaData_DS['description'].astype(str) + ' ' + \
                               MetaData_DS['summary'].astype(str) + ' ' + \
                               MetaData_DS['augmented_tags'].astype(str)

# Tokenize dataset descriptions
tokenized_text = tokenizer(list(MetaData_DS['combined_text']), truncation=True, padding=True, max_length=max_length)

# Convert labels to numerical format
labels = [1 if url else 0 for url in MetaData_DS['dataset_url']]

# Extract input_ids and attention_mask
input_ids = tokenized_descriptions['input_ids']
attention_mask = tokenized_descriptions['attention_mask']

# Split data into training and validation sets
train_ids, val_ids, train_masks, val_masks, train_labels, val_labels = train_test_split(input_ids, attention_mask, labels, test_size=0.2, random_state=42)

# Convert tokenized texts and labels to PyTorch tensors
train_ids = torch.tensor(train_ids)
val_ids = torch.tensor(val_ids)
train_masks = torch.tensor(train_masks)
val_masks = torch.tensor(val_masks)
train_labels = torch.tensor(train_labels)
val_labels = torch.tensor(val_labels)

# Create TensorDataset for training and validation sets
train_dataset = TensorDataset(train_ids, train_masks, train_labels)
val_dataset = TensorDataset(val_ids, val_masks, val_labels)

# Define batch size
batch_size = 16

# Create DataLoader for training and validation sets
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Define the BERT-based model with increased dropout
class BERTClassifier(nn.Module):
    def __init__(self, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased', hidden_dropout_prob=0.5, attention_probs_dropout_prob=0.5)  # Further increased BERT dropout
        self.dropout = nn.Dropout(0.5)  # Further increased dropout rate
        self.linear = nn.Linear(self.bert.config.hidden_size, num_classes)
    
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        logits = self.linear(pooled_output)
        return logits

# Instantiate the BERT model
num_classes = 2  
model = BERTClassifier(num_classes)

# Define the optimizer with weight decay and gradient clipping
optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=1, verbose=True)
criterion = nn.CrossEntropyLoss()

# Training loop with early stopping
epochs = 10
patience = 2
best_loss = float('inf')
trigger_times = 0

for epoch in range(epochs):
    model.train()  # Set the model to training mode
    total_loss = 0
    total_correct = 0
    total_samples = 0
    for batch in train_loader:
        inputs, attention_masks, labels = batch

        # Clear gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs, attention_masks)
        # Compute loss
        loss = criterion(outputs, labels)

        # Backpropagation
        loss.backward()

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        # Update model parameters
        optimizer.step()

        # Accumulate the total loss
        total_loss += loss.item()

        # Calculate accuracy for this batch
        _, predicted = torch.max(outputs, dim=1)
        total_correct += (predicted == labels).sum().item()
        total_samples += labels.size(0)

    avg_train_loss = total_loss / len(train_loader)
    train_accuracy = total_correct / total_samples

    # Validation loop
    model.eval()  # Set the model to evaluation mode
    val_loss = 0
    total_correct = 0
    total_samples = 0
    all_predictions = []
    all_labels = []
    with torch.no_grad():
        for batch in val_loader:
            inputs, attention_masks, labels = batch

            # Forward pass
            outputs = model(inputs, attention_masks)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            
            # Compute predictions
            _, predicted = torch.max(outputs, dim=1)

            # Update total samples and total correct predictions
            total_samples += labels.size(0)
            total_correct += (predicted == labels).sum().item()

            # Save predictions and labels for metric calculation
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_val_loss = val_loss / len(val_loader)
    accuracy = total_correct / total_samples
    scheduler.step(avg_val_loss)
    
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_predictions, average='binary')

    print(f"Epoch {epoch + 1}/{epochs}")
    print(f"Train Loss: {avg_train_loss}, Train Accuracy: {train_accuracy}")
    print(f"Validation Loss: {avg_val_loss}, Validation Accuracy: {accuracy}")
    print(f"Validation Precision: {precision}, Validation Recall: {recall}, Validation F1 Score: {f1}")

    # Early stopping
    if avg_val_loss < best_loss:
        best_loss = avg_val_loss
        trigger_times = 0
    else:
        trigger_times += 1
        if trigger_times >= patience:
            print('Early stopping!')
            break


Epoch 1/10
Train Loss: 0.607300654053688, Train Accuracy: 0.6825396825396826
Validation Loss: 0.6546359062194824, Validation Accuracy: 0.625
Validation Precision: 1.0, Validation Recall: 0.625, Validation F1 Score: 0.7692307692307693
Epoch 2/10
Train Loss: 0.39308468997478485, Train Accuracy: 0.9365079365079365
Validation Loss: 0.3837055265903473, Validation Accuracy: 1.0
Validation Precision: 1.0, Validation Recall: 1.0, Validation F1 Score: 1.0
Epoch 3/10
Train Loss: 0.25504642724990845, Train Accuracy: 0.9841269841269841
Validation Loss: 0.24229596555233002, Validation Accuracy: 1.0
Validation Precision: 1.0, Validation Recall: 1.0, Validation F1 Score: 1.0
Epoch 4/10
Train Loss: 0.17578090727329254, Train Accuracy: 1.0
Validation Loss: 0.17643404006958008, Validation Accuracy: 1.0
Validation Precision: 1.0, Validation Recall: 1.0, Validation F1 Score: 1.0
Epoch 5/10
Train Loss: 0.12218410149216652, Train Accuracy: 1.0
Validation Loss: 0.13565519452095032, Validation Accuracy: 1.0
V